<a href="https://colab.research.google.com/github/Jeff-Delavusca/Econometrics-and-ML/blob/main/modelo_var.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importação de dados e bilbiotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller, kpss, coint


## Conhecendo a base de dados

A base de dados contém informações sobre a data, que inicia em janeiro de 2003 e encerra em agosto de 2025. Além disso, tem as seguintes informações:


*   **taxa_juros_real**: refere-se a taxa de juros swap de 360 dias - expectativas de juros futuros - corrigida pelas expectativas de inflação para 12 meses.
*   **bens_capital**: refere-se ao índice de produção de bens de capital.
*   **bens_intermediarios**: refere-se ao índice de produção de bens intermediários.
*   **bens_duraveis**: refere-se ao índice de produção de bens de consumo duráveis.
*   **ibc**: refere-se ao índice de produção industrial do Brail (proxy do PIB).
*   **cambio_refetivo**: refere-se a taxa de câmbio real efetiva.
*   **tjlp**: refere-se a taxa de juros de longo prazo.

Todas as variáveis está com uma frequência mensal.





In [ ]:
caminho = '/content/base_ajustada_com_selic.xlsx'

dados = pd.read_excel(caminho)
print("Bibliotecas importadas e base de dados pronta.\n")
display(dados.head())

## Objetivo

O objetivo da pesquisa é mensurar o efeito de uma alteração da política monetária sobre a produção industrial através dos grandes setores econômicos. Usou-se o conceito de efeito pass-through da macroeconomia para auxliar na interpretação econômica.

# Análise exploratória de dados

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(20, 5))  # 4 linhas, 2 colunas

# Exemplo de preenchimento dos 8 gráficos:
axs[0, 0].plot(dados['data'], dados['selic_acm_12_meses'])
axs[0, 1].plot(dados['data'], dados['bens_capital'])
axs[1, 0].plot(dados['data'], dados['bens_intermediarios'])
axs[1, 1].plot(dados['data'], dados['bens_consumo'])


# Títulos (exemplo)
axs[0, 0].set_title('Taxa de Juros Selic Acumulado 12 meses')
axs[0, 1].set_title('Índice de Produção de Bens de Capital (Sazonalizado)')
axs[1, 0].set_title('Índice de Produção de Bens Intermediário (Sazonalizados)')
axs[1, 1].set_title('Índice de Produão de Bens de Consumo Duráveis e Não Duráveis (Sazonalizados)')



plt.tight_layout()


# Teste ADF e gráficos de Autocorrelação e Autocorrelação Parcial em nível

Para avaliar a presença de raiz unitária nas séries temporais utilizadas na modelagem econométrica, foi empregado o teste de Dickey-Fuller Aumentado (ADF), conforme proposto por Dickey e Fuller (1979) e posteriormente expandido por Said e Dickey (1984). O objetivo do teste é verificar se a série apresenta comportamento estacionário ou se possui raiz unitária, condição que inviabiliza estimações tradicionais e pode gerar regressões espúrias (GUJARATI; PORTER, 2011).

Considerando que as séries macroeconômicas analisadas (como PIB, IBC-Br, produção industrial e índices de preço) apresentam tendência determinística ao longo do tempo, o teste foi estimado na forma que inclui intercepto e tendência determinística, especificada no parâmetro regression='ct'. Essa escolha é consistente com a literatura, que recomenda o uso do modelo com tendência em séries que exibem crescimento ou queda sistemática ao longo do período (ENDERS, 2015).

In [ ]:
# Função para realizar os testes

def realizar_testes_estacionariedade(serie):
    # --- Teste ADF ---
    adf_result = adfuller(serie, autolag="AIC")
    print("--- Teste ADF ---")
    print(f"Estatística ADF: {adf_result[0]:.4f}")
    print(f"P-valor: {adf_result[1]:.4f}")
    print(f"Estacionária (P-valor < 0.05)? {'SIM' if adf_result[1] < 0.05 else 'NÃO'}")

    # --- Teste KPSS (H0: Estacionária) ---
    # Usamos 'ct' (com tendência e intercepto) para a maioria das séries econômicas
    kpss_result = kpss(serie, regression='ct')
    print("\n--- Teste KPSS ---")
    print(f"Estatística KPSS: {kpss_result[0]:.4f}")
    print(f"P-valor: {kpss_result[1]:.4f}")
    # Nota: No KPSS, rejeitar H0 (P-valor < 0.05) significa que NÃO é estacionária.
    print(f"Estacionária (P-valor > 0.05)? {'SIM' if kpss_result[1] > 0.05 else 'NÃO'}")

# Exemplo de execução:
# realizar_testes_estacionariedade(dados['d_ln_bens_capital'].dropna())

In [ ]:
realizar_testes_estacionariedade(dados['selic_acm_12_meses'].dropna())

In [ ]:
# 1. Aplica as transformações criando novas colunas no DataFrame 'dados'
dados['d_ln_bens_capital'] = np.log(dados['bens_capital']).diff()
dados['d_ln_bens_intermediarios'] = np.log(dados['bens_intermediarios']).diff()
dados['d_ln_bens_consumo'] = np.log(dados['bens_consumo']).diff()
dados['d_taxa_selic_acm_12_meses'] = (dados['selic_acm_12_meses']).diff()

# 2. Define a lista de colunas que farão parte do modelo VAR
# Lembre-se que a primeira observação (NaN após o .diff()) será excluída automaticamente pelo VAR.
colunas_finais = ['data',
    'd_ln_bens_capital',
    'd_ln_bens_intermediarios',
    'd_ln_bens_consumo',
    'd_taxa_selic_acm_12_meses'
]

# 3. Cria um novo DataFrame apenas com as colunas transformadas (e descartando NaNs)
df_diff = dados[colunas_finais].dropna()

# Exibe as primeiras linhas do DataFrame final para checagem
display(df_diff.head())

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(20, 5))  # 4 linhas, 2 colunas

# Exemplo de preenchimento dos 8 gráficos:
axs[0, 0].plot(df_diff['data'], df_diff['d_taxa_selic_acm_12_meses'])
axs[0, 1].plot(df_diff['data'], df_diff['d_ln_bens_capital'])
axs[1, 0].plot(df_diff['data'], df_diff['d_ln_bens_intermediarios'])
axs[1, 1].plot(df_diff['data'], df_diff['d_ln_bens_consumo'])


# Títulos (exemplo)
axs[0, 0].set_title('Taxa de Juros Selic Acumulado 12 meses')
axs[0, 1].set_title('Índice de Produção de Bens de Capital (Sazonalizado)')
axs[1, 0].set_title('Índice de Produção de Bens Intermediário (Sazonalizados)')
axs[1, 1].set_title('Índice de Produão de Bens de Consumo Duráveis e Não Duráveis (Sazonalizados)')

plt.tight_layout()

In [ ]:
realizar_testes_estacionariedade(df_diff['d_taxa_selic_acm_12_meses'].dropna())

In [ ]:
from statsmodels.tsa.api import VAR

# 1. Inicialize e execute a seleção de ordem para o MODELO SEM CONSTANTE INTERCEPTO (n)
print("Modelo sem Constante intercepto (trend='n')")
# Assume-se que 'df_ordenado' é o seu DataFrame I(0) de 4 variáveis
base_var = df_diff[['d_taxa_selic_acm_12_meses', 'd_ln_bens_capital', 'd_ln_bens_intermediarios', 'd_ln_bens_consumo']]
# Criar um modelo VAR
model_n = VAR(base_var)
results_n = model_n.select_order(maxlags=12, trend='n')
print(results_n.summary())
# Access the BIC values from the 'selected_orders' attribute
lag_bic_n = results_n.selected_orders['aic']
print(f"\nLag sugerido pelo BIC (sem Constante Intercepto): {lag_bic_n}")



# 2. Inicialize e execute a seleção de ordem para o MODELO COM CONSTANTE INTERCEPTO (c)
print("\nModelo com Constante Intercepto(trend='c')")
model_c = VAR(base_var)
results_c = model_c.select_order(maxlags=12, trend='c')
print(results_c.summary())
# Access the BIC values from the 'selected_orders' attribute for the no-constant model
lag_bic_c = results_c.selected_orders['aic']
print(f"\nLag sugerido pelo AIC (com Constante Intercepto): {lag_bic_c}")


In [ ]:
from statsmodels.tsa.api import VAR

# 1. Inicializa o modelo VAR
model = VAR(base_var)

# 2. Estima o modelo VAR(4)
lag_order = 4
results_var4 = model.fit(lag_order, trend='c')

# 3. Teste de Autocorrelação dos Resíduos (Portmanteau)
# H_0: Não há autocorrelação. O P-valor deve ser > 0.05.
#print("\n--- Teste de Autocorrelação (Portmanteau) VAR(3) ---")
print(results_var4.test_whiteness(nlags=10).summary())

In [ ]:
'''
from statsmodels.tsa.api import VAR

# Estimar modelo
model = VAR(base_var)
results = model.fit(4)

# Previsão 5 passos
lag_order = results.k_ar
forecast_input = base_var.values[-lag_order:]

forecast = results.forecast(forecast_input, steps=5)
print(forecast)

# Plot
results.plot_forecast(10)

'''

In [ ]:
'''residuos = results_var4.resid

for col in residuos.columns:
    print(f"\nLjung-Box para: {col}")
    print(acorr_ljungbox(residuos[col], lags=[10], return_df=True))'''

In [ ]:
results_var4.summary()

In [ ]:
# H0: d_taxa_selic_acm_12_meses NÃO Granger-causa os três setores conjuntamente.
#print("\n## 1. Teste: Selic acumulada 12 meses -> Produção de Bens Intermediários")
granger_eficacia = results_var4.test_causality(
    [
        'd_ln_bens_consumo'
    ], # Variáveis Resposta (Quem é previsto)
    ['d_taxa_selic_acm_12_meses'], # Variável Causando (Juro)
    kind='wald' # Teste Wald para restrições conjuntas
)
print(granger_eficacia.summary())

In [ ]:

'''
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
import numpy as np

# 1. Calcule o IRF para 24 meses
irf = results_var4.irf(periods=24)

# 2. Defina impulso e respostas
impulso_interesse = 'd_taxa_selic_acm_12_meses'
respostas_interesse = ['d_ln_bens_capital', 'd_ln_bens_intermediarios', 'd_ln_bens_consumo']

# 3. Intervalos de confiança via simulação Monte Carlo
lower, upper = irf.errband_mc(orth=True, repl=1000, signif=0.05)

# 4. Criação dos gráficos
fig, axes = plt.subplots(nrows=len(respostas_interesse), ncols=1, figsize=(10, 4 * len(respostas_interesse)))

if len(respostas_interesse) == 1:
    axes = [axes]

impulse_idx = base_var.columns.get_loc(impulso_interesse)

for i, response_var in enumerate(respostas_interesse):

    response_idx = base_var.columns.get_loc(response_var)

    # IRF principal
    irf_values = irf.irfs[:, response_idx, impulse_idx]

    # Intervalos inferior e superior
    lower_ci = lower[:, response_idx, impulse_idx]
    upper_ci = upper[:, response_idx, impulse_idx]

    # Plot da curva principal
    axes[i].plot(range(len(irf_values)), irf_values, label=response_var, color='blue')

    # Plot do intervalo de confiança (banda cinza)
    axes[i].fill_between(range(len(irf_values)), lower_ci, upper_ci, color='gray', alpha=0.3)

    axes[i].axhline(0, color='black', linestyle='--')
    axes[i].set_title(f'Resposta de {response_var} a um choque em {impulso_interesse}')
    axes[i].set_xlabel('Período')
    axes[i].set_ylabel('Magnitude da Resposta')
    axes[i].legend()

fig.suptitle('Respostas Setoriais a um Choque de Juro Real (VAR - Cholesky)', fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()
'''


In [ ]:
# 1. Calcule o FIR com Bandas de Confiança (rigor estatístico)
impulso_interesse = 'd_taxa_selic_acm_12_meses'
irf = results_var4.irf(periods=10)

# 2. Plotagem simplificada (Usando o método de plotagem nativo)
# O método plot() do objeto irf é mais fácil de usar para mostrar as bandas.

'''
# Loop through each response variable to plot individually
for response_var in respostas_interesse:
    fig = irf.plot(
        impulse=impulso_interesse,
        #response=response_var, # Plot each response variable individually
        plot_params={'figsize': (5, 4)}, # Define o tamanho da figura
        signif=0.05,
        orth=True # Garante que está usando o IRF ortogonalizado (Cholesky)
    )
    fig.suptitle(f'Resposta de {response_var} a um Choque de {impulso_interesse} (Com Bandas de 95%)', fontsize=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.9])
    plt.show()
    '''
fig = irf.plot(
impulse=impulso_interesse,
response='d_ln_bens_capital', # Plot each response variable individually
#plot_params={'figsize': (5, 4)}, # Define o tamanho da figura
signif=0.05,
orth=True # Garante que está usando o IRF ortogonalizado (Cholesky)
)


fig.set_size_inches(6, 4)  # <- diminui o tamanho aqui
fig.suptitle(f'Resposta de Bens de Capital a um Choque da Taxa Selic (Com Bandas de 95%)', fontsize=12)

plt.tight_layout(rect=[0, 0.03, 1, 0.9])
plt.show()

In [ ]:
'''
# 1. Gera o objeto IRF para 24 períodos (ou o que você estiver usando)
irf = results_var4.irf(periods=24)

# 2. Encontra os índices de impulso e resposta
impulse_idx = base_var.columns.get_loc('d_taxa_selic_acm_12_meses')
response_idx = base_var.columns.get_loc('d_ln_bens_consumo')

# 3. Extrai manualmente os valores do IRF
irf_values = irf.irfs[:, response_idx, impulse_idx]

# 4. Encontra o menor valor e o período em que ele ocorre
min_value = irf_values.min()
min_period = irf_values.argmin()

print("Menor valor:", min_value)
print("Período onde ocorre:", min_period)

# 5. Plot usando o método padrão da biblioteca
fig = irf.plot(
    impulse='d_taxa_selic_acm_12_meses',
    response='d_ln_bens_consumo',
    signif=0.05,
    orth=True
)

fig.set_size_inches(6, 4)
fig.suptitle('Resposta de Bens de Capital a um Choque da Taxa Selic (Com Bandas de 95%)', fontsize=12)

# 6. Marca o ponto mínimo no gráfico
ax = fig.axes[0]   # primeiro subplot
ax.scatter(min_period, min_value, color='red', zorder=5)
ax.annotate(
    f'Mínimo = {min_value:.4f}\nPeríodo = {min_period}',
    (min_period, min_value),
    textcoords="offset points",
    xytext=(10, -10),
    ha='left',
    fontsize=8,
    color='red'
)

plt.tight_layout(rect=[0, 0.03, 1, 0.9])
plt.show()
'''

In [ ]:
fig = irf.plot(
impulse=impulso_interesse,
response='d_ln_bens_intermediarios', # Plot each response variable individually
#plot_params={'figsize': (5, 4)}, # Define o tamanho da figura
signif=0.05,
orth=True # Garante que está usando o IRF ortogonalizado (Cholesky)
)


fig.set_size_inches(6, 4)  # <- diminui o tamanho aqui
fig.suptitle(f'Resposta de Bens Intermediários a um Choque da Taxa Selic (Com Bandas de 95%)', fontsize=12)

plt.tight_layout(rect=[0, 0.03, 1, 0.9])
plt.show()

In [ ]:
fig = irf.plot(
impulse=impulso_interesse,
response='d_ln_bens_consumo', # Plot each response variable individually
#plot_params={'figsize': (5, 4)}, # Define o tamanho da figura
signif=0.05,
orth=True # Garante que está usando o IRF ortogonalizado (Cholesky)
)


fig.set_size_inches(6, 4)  # <- diminui o tamanho aqui
fig.suptitle(f'Resposta de Bens de Consumo a um Choque da Taxa Selic (Com Bandas de 95%)', fontsize=12)

plt.tight_layout(rect=[0, 0.03, 1, 0.9])
plt.show()

In [ ]:
fevd = results_var4.fevd(36)
fevd.summary()

In [ ]:
results_var4.fevd(10).plot()